### 추정교통량 + 혼잡빈도강도 + 혼잡시간강도

In [1]:
import pathlib
import pandas as pd
import geopandas as gpd
import json

In [2]:
import folium
#from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.cluster import KMeans
import seaborn as sns
import shapely
import pydeck as pdk

In [3]:
input_path = pathlib.Path('./data')
주정차단속 = pd.read_csv(input_path.joinpath('1.오산시_주정차단속(2018~2020).csv'),encoding="utf-8")
어린이교통사고_격자 = gpd.read_file(input_path.joinpath('2.오산시_어린이교통사고_격자.geojson'),encoding="utf-8")
차량등록현황_격자 = gpd.read_file(input_path.joinpath('3.오산시_차량등록현황_격자.geojson'),encoding="utf-8")
거주인구격자_총인구 = gpd.read_file(input_path.joinpath('4.오산시_연령별_거주인구격자(총인구).geojson'),encoding="utf-8")
거주인구격자_유소년 = gpd.read_file(input_path.joinpath('5.오산시_연령별_거주인구격자(유소년).geojson'),encoding="utf-8")
거주인구격자_생산가능 = gpd.read_file(input_path.joinpath('6.오산시_연령별_거주인구격자(생산가능인구).geojson'),encoding="utf-8")
거주인구격자_고령 = gpd.read_file(input_path.joinpath('7.오산시_연령별_거주인구격자(고령).geojson'),encoding="utf-8")
유동인구 = pd.read_csv(input_path.joinpath('8.오산시_유동인구(2019).csv'),encoding="utf-8")
어린이보호구역 = pd.read_csv(input_path.joinpath('9.오산시_어린이보호구역.csv'),encoding="utf-8")
학교위치정보 = pd.read_csv(input_path.joinpath('10.오산시_학교위치정보.csv'),encoding="utf-8")
초등학교_통학구 = gpd.read_file(input_path.joinpath('11.오산시_초등학교_통학구.geojson'),encoding="utf-8")
중학교_학군 = gpd.read_file(input_path.joinpath('12.오산시_중학교_학군.geojson'),encoding="utf-8")
어린이집_유치원현황 = pd.read_csv(input_path.joinpath('13.오산시_어린이집_유치원현황.csv'),encoding="utf-8")
기상데이터 = pd.read_csv(input_path.joinpath('14.오산시_기상데이터(2010~2019).csv'),encoding="utf-8")
무인교통단속카메라 = pd.read_csv(input_path.joinpath('15.오산시_무인교통단속카메라.csv'),encoding="utf-8")
도로안전표지 = pd.read_csv(input_path.joinpath('16.오산시_도로안전표지표준데이터.csv'),encoding="utf-8")
횡단보도 = gpd.read_file(input_path.joinpath('17.오산시_횡단보도.geojson'),encoding="utf-8")
과속방지턱 = pd.read_csv(input_path.joinpath('18.오산시_과속방지턱표준데이터.csv'),encoding="utf-8")
신호등 = gpd.read_file(input_path.joinpath('19.오산시_신호등.geojson'),encoding="utf-8")
CCTV설치현황 = pd.read_csv(input_path.joinpath('20.오산시_CCTV설치현황.csv'),encoding="utf-8")
인도 = gpd.read_file(input_path.joinpath('21.오산시_인도.geojson'),encoding="utf-8")
버스정류장 = pd.read_csv(input_path.joinpath('22.오산시_버스정류장.csv'),encoding="utf-8")
상세도로망 = gpd.read_file(input_path.joinpath('23.오산시_상세도로망_LV6.geojson'),encoding="utf-8")
시간대별_추정교통량 = pd.read_csv(input_path.joinpath('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv'),encoding="utf-8")
혼잡빈도강도 = pd.read_csv(input_path.joinpath('25.평일_전일_오산시_혼잡빈도강도_Level6.csv'),encoding="utf-8")
혼잡시간강도 = pd.read_csv(input_path.joinpath('26.평일_전일_오산시_혼잡시간강도_Level6.csv'),encoding="utf-8")
도로명주소_건물 = gpd.read_file(input_path.joinpath('27.오산시_도로명주소_건물.geojson'),encoding="utf-8")
건물연면적_격자 = gpd.read_file(input_path.joinpath('28.오산시_건물연면적_격자.geojson'),encoding="utf-8")
체육시설현황 = pd.read_csv(input_path.joinpath('29.오산시_체육시설현황.csv'),encoding="utf-8")
학원_및_교습소_현황 = pd.read_csv(input_path.joinpath('30.오산시_학원_및_교습소_현황.csv'),encoding="utf-8")
법정경계_시군구 = gpd.read_file(input_path.joinpath('31.오산시_법정경계(시군구).geojson'),encoding="utf-8")
행정경계_읍면동 = gpd.read_file(input_path.joinpath('32.오산시_행정경계(읍면동).geojson'),encoding="utf-8")
법정경계_읍면동 = gpd.read_file(input_path.joinpath('33.오산시_법정경계(읍면동).geojson'),encoding="utf-8")
지적도 = gpd.read_file(input_path.joinpath('34.오산시_지적도.geojson'),encoding="utf-8")
코드정의서 =  pd.read_excel(input_path.joinpath('35.코드정의서.xlsx'))

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- 24.시간대별 추정교통량 (24번 data)

In [4]:
교통량 = 시간대별_추정교통량
교통량.info() #81975행
교통량.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81975 entries, 0 to 81974
Data columns (total 13 columns):
상세도로망_LinkID    81975 non-null int64
도로등급            81975 non-null int64
링크길이            81975 non-null float64
도로명             69075 non-null object
시도명             81975 non-null object
시군구명            81975 non-null object
읍면동명            81975 non-null object
시간적범위           81975 non-null object
평일주말            81975 non-null object
전체 추정교통량        81975 non-null float64
승용차 추정교통량       81975 non-null float64
버스 추정교통량        81975 non-null float64
화물차 추정교통량       81975 non-null float64
dtypes: float64(5), int64(2), object(6)
memory usage: 8.1+ MB


,상세도로망_LinkID,도로등급,링크길이,전체 추정교통량,승용차 추정교통량,버스 추정교통량,화물차 추정교통량
count,8.197500e+04,81975.000000,81975.000000,81975.000000,81975.000000,81975.000000,81975.000000
mean,5.712641e+10,106.608112,0.120636,483.963280,405.490657,8.755831,69.716644
std,9.326520e+08,1.101092,0.153740,2069.872552,1676.803712,66.802921,347.093994
min,4.783447e+10,101.000000,0.004000,0.000000,0.000000,0.000000,0.000000
25%,5.712450e+10,107.000000,0.036000,24.470000,20.850000,0.030000,1.450000
50%,5.712523e+10,107.000000,0.074000,96.340000,83.780000,0.360000,10.600000
75%,5.713539e+10,107.000000,0.153000,357.700000,306.990000,3.320000,47.295000
max,9.000002e+10,108.000000,3.171000,106893.770000,80493.300000,5156.780000,21275.900000


In [5]:
교통량.시간적범위.value_counts() ## 왜이렇게 나오지??? >> type이 str/ int 섞임

전일    3279
10    3277
11    3277
1     3277
19    3277
18    3277
14    3277
17    3277
15    3277
12    3277
13    3277
2     3277
0     3277
16    3277
8     2600
4     2070
7     2070
6     2070
3     2070
5     2070
23    1989
9     1989
22    1989
21    1989
20    1733
20    1546
9     1290
21    1290
22    1290
23    1290
3     1209
4     1209
5     1209
6     1209
7     1209
8      679
1        2
2        2
16       2
10       2
11       2
12       2
13       2
14       2
15       2
17       2
18       2
19       2
0        2
Name: 시간적범위, dtype: int64

In [6]:
교통량.시간적범위[교통량.시간적범위!='전일'] = 교통량.시간적범위[교통량.시간적범위!='전일'].map(lambda x : int(x))

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
교통량.시간적범위.value_counts() # 교통량 전처리 완료

전일      3279
11.0    3279
1.0     3279
2.0     3279
3.0     3279
4.0     3279
5.0     3279
6.0     3279
7.0     3279
8.0     3279
9.0     3279
10.0    3279
12.0    3279
23.0    3279
13.0    3279
14.0    3279
15.0    3279
16.0    3279
17.0    3279
18.0    3279
19.0    3279
20.0    3279
21.0    3279
22.0    3279
0.0     3279
Name: 시간적범위, dtype: int64

In [9]:
교통량.isnull().sum() # 도로명만 nan값 존재

상세도로망_LinkID        0
도로등급                0
링크길이                0
도로명             12900
시도명                 0
시군구명                0
읍면동명                0
시간적범위               0
평일주말                0
전체 추정교통량            0
승용차 추정교통량           0
버스 추정교통량            0
화물차 추정교통량           0
dtype: int64

- 26. 혼잡시간강도 (26번 data)

In [14]:
혼잡시간강도.head() 

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,시간적범위,평일주말,혼잡시간강도
0,57125499201,101,1.238,경부고속도로,경기도,오산시,대원동,전일,평일,59.66
1,57125499001,101,1.335,경부고속도로,경기도,오산시,대원동,전일,평일,77.13
2,57137697901,101,0.093,수도권제2순환고속도로(봉담동탄,경기도,오산시,세마동,전일,평일,50.79
3,57135249601,101,0.324,수도권제2순환고속도로(봉담동탄,경기도,오산시,세마동,전일,평일,99.75
4,57135250601,101,0.333,수도권제2순환고속도로(봉담동탄,경기도,오산시,세마동,전일,평일,99.99


In [15]:
혼잡시간강도.isnull().sum() #도로명만 nan값 존재

상세도로망_LinkID      0
도로등급              0
링크길이              0
도로명             382
시도명               0
시군구명              0
읍면동명              0
시간적범위             0
평일주말              0
혼잡시간강도            0
dtype: int64

- 도로명 null 값 382개
---------------------------------------------------------------------------------------------------------------------------

- 25.혼잡빈도강도 (25번 데이터)

In [16]:
혼잡빈도강도.head() 

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,시간적범위,평일주말,혼잡빈도강도
0,57125499201,101,1.238,경부고속도로,경기도,오산시,대원동,전일,평일,49.10
1,57125499001,101,1.335,경부고속도로,경기도,오산시,대원동,전일,평일,70.67
2,57137697901,101,0.093,수도권제2순환고속도로(봉담동탄,경기도,오산시,세마동,전일,평일,42.61
3,57135249601,101,0.324,수도권제2순환고속도로(봉담동탄,경기도,오산시,세마동,전일,평일,96.75
4,57135250601,101,0.333,수도권제2순환고속도로(봉담동탄,경기도,오산시,세마동,전일,평일,99.95


In [17]:
혼잡빈도강도.isnull().sum() # 도로명만 nan값 존재

상세도로망_LinkID      0
도로등급              0
링크길이              0
도로명             382
시도명               0
시군구명              0
읍면동명              0
시간적범위             0
평일주말              0
혼잡빈도강도            0
dtype: int64

- 시간강도와 동일하게 도로명 null 값 382개

- 23.상세도로망 (23번data)

In [18]:
상세도로망.isnull().sum()

link_id          0
max_speed        0
road_name      294
road_no          0
road_rank        0
link_type        0
road_type        0
facil_name    1842
up_lanes         0
dw_lanes         0
oneway           0
length           0
width            0
car_lane         0
num_cross        0
barrier          0
geometry         0
dtype: int64

In [19]:
## Merge data

In [20]:
# 시간범위가 1시간 단위가 아닌 '전일' 만
교통량_전일 = 교통량[교통량['시간적범위'] == '전일']
교통량_전일.info()
교통량.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3279 entries, 577 to 11384
Data columns (total 13 columns):
상세도로망_LinkID    3279 non-null int64
도로등급            3279 non-null int64
링크길이            3279 non-null float64
도로명             2763 non-null object
시도명             3279 non-null object
시군구명            3279 non-null object
읍면동명            3279 non-null object
시간적범위           3279 non-null object
평일주말            3279 non-null object
전체 추정교통량        3279 non-null float64
승용차 추정교통량       3279 non-null float64
버스 추정교통량        3279 non-null float64
화물차 추정교통량       3279 non-null float64
dtypes: float64(5), int64(2), object(6)
memory usage: 358.6+ KB


(81975, 13)

### 전일 기준으로 24(추정교통량) + 25,26(혼잡시간강도, 혼잡빈도강도) merge

In [21]:
혼잡시간강도_전일 = 혼잡시간강도[['상세도로망_LinkID','혼잡시간강도']]
혼잡빈도강도_전일 = 혼잡빈도강도[['상세도로망_LinkID','혼잡빈도강도']]
혼잡빈도강도.shape

(3032, 10)

In [22]:
교통량_merge_temp = 교통량_전일.merge(혼잡시간강도_전일,on='상세도로망_LinkID',how='outer')
교통량_merge = 교통량_merge_temp.merge(혼잡빈도강도_전일,on='상세도로망_LinkID',how='outer')
교통량_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3279 entries, 0 to 3278
Data columns (total 15 columns):
상세도로망_LinkID    3279 non-null int64
도로등급            3279 non-null int64
링크길이            3279 non-null float64
도로명             2763 non-null object
시도명             3279 non-null object
시군구명            3279 non-null object
읍면동명            3279 non-null object
시간적범위           3279 non-null object
평일주말            3279 non-null object
전체 추정교통량        3279 non-null float64
승용차 추정교통량       3279 non-null float64
버스 추정교통량        3279 non-null float64
화물차 추정교통량       3279 non-null float64
혼잡시간강도          3032 non-null float64
혼잡빈도강도          3032 non-null float64
dtypes: float64(7), int64(2), object(6)
memory usage: 409.9+ KB


- 마지막 2행에서 연결잘됨 확인(13,14)

----------------------------------------------------------------------------------------------------

### Merge table + 혼잡시간차량/혼잡빈도차량
- 계산을 통해 혼잡시간을 경헙한 차량의 수와 혼잡빈도를 경험한 차량의 수를 수치에서 직접적인 차량의 수로 정량화 시킴
- 그 후 merge table에 파생변수 column으로 추가

In [23]:
# 혼잡 경험한 지나가는 차량 대수 >> 교통량 * (시간or 빈도)강도/100  [혼잡시간_car/혼잡빈도_car]
교통량_merge['혼잡시간_car'] = 교통량_merge['전체 추정교통량'] * (교통량_merge['혼잡시간강도']/100)
교통량_merge['혼잡빈도_car'] = 교통량_merge['전체 추정교통량'] * (교통량_merge['혼잡빈도강도']/100)
교통량_merge = 교통량_merge.drop(['시도명','시군구명','시간적범위','평일주말'], axis =1)
교통량_merge.head()

,상세도로망_LinkID,도로등급,링크길이,도로명,읍면동명,전체 추정교통량,승용차 추정교통량,버스 추정교통량,화물차 추정교통량,혼잡시간강도,혼잡빈도강도,혼잡시간_car,혼잡빈도_car
0,57125499201,101,1.238,경부고속도로,대원동,104794.49,78556.90,5128.22,21109.36,59.66,49.10,62520.392734,51454.094590
1,57125499001,101,1.335,경부고속도로,대원동,101620.94,76522.02,5156.78,19942.14,77.13,70.67,78380.231022,71815.518298
2,57137697901,101,0.093,수도권제2순환고속도로(봉담동탄,세마동,30228.45,22684.54,958.95,6584.96,50.79,42.61,15353.029755,12880.342545
3,57135249601,101,0.324,수도권제2순환고속도로(봉담동탄,세마동,2219.96,1667.75,80.49,471.71,99.75,96.75,2214.410100,2147.811300
4,57135250601,101,0.333,수도권제2순환고속도로(봉담동탄,세마동,1848.42,1270.28,47.78,530.35,99.99,99.95,1848.235158,1847.495790


#### 주의!
- 12 기준  (11~12)시
- 13 기준부터 (13~14)시
    - (12~13)시 데이터는 없음


## 23(상세도로망) + 24,25,26(혼잡시간/혼잡빈도/추정교통량) 최종 merge
#### 상세도로망 분리 ( 상행과 하행)
- 01 상행 /  02 하행
- 01은 상행만 있고, 02는 상/하행 다 있음 >> 하행만 있는 경우는 없다

In [29]:
교통량_merge.상세도로망_LinkID.head()

0    57125499201
1    57125499001
2    57137697901
3    57135249601
4    57135250601
Name: 상세도로망_LinkID, dtype: object

In [30]:
교통량_merge['상세도로망_LinkID'] = 교통량_merge['상세도로망_LinkID'].apply(str)

In [31]:
# 총 11자리 끝에 01/02 분류
# 상 하행 분리해서 merge table에 상/하행 파생변수 column 추가
교통량_merge['link_id'] = 교통량_merge['상세도로망_LinkID'].str[0:9]
교통량_merge['상하행'] = 교통량_merge['상세도로망_LinkID'][:].str[9:11].replace({'01':'상행','02':'하행'})

In [32]:
교통량_merge.head()

,상세도로망_LinkID,도로등급,링크길이,도로명,읍면동명,전체 추정교통량,승용차 추정교통량,버스 추정교통량,화물차 추정교통량,혼잡시간강도,혼잡빈도강도,혼잡시간_car,혼잡빈도_car,link_id,상하행
0,57125499201,101,1.238,경부고속도로,대원동,104794.49,78556.90,5128.22,21109.36,59.66,49.10,62520.392734,51454.094590,571254992,상행
1,57125499001,101,1.335,경부고속도로,대원동,101620.94,76522.02,5156.78,19942.14,77.13,70.67,78380.231022,71815.518298,571254990,상행
2,57137697901,101,0.093,수도권제2순환고속도로(봉담동탄,세마동,30228.45,22684.54,958.95,6584.96,50.79,42.61,15353.029755,12880.342545,571376979,상행
3,57135249601,101,0.324,수도권제2순환고속도로(봉담동탄,세마동,2219.96,1667.75,80.49,471.71,99.75,96.75,2214.410100,2147.811300,571352496,상행
4,57135250601,101,0.333,수도권제2순환고속도로(봉담동탄,세마동,1848.42,1270.28,47.78,530.35,99.99,99.95,1848.235158,1847.495790,571352506,상행


In [33]:
교통량_merge.상하행.value_counts()

상행    1838
하행    1441
Name: 상하행, dtype: int64

- 여태까지의 merge table 바탕으로 link_id와 읍면동명으로 grouping

In [34]:
# 지역별 평균 전체 추정교통량
교통량_group = 교통량_merge.groupby(['link_id','읍면동명'],as_index=False).mean()
교통량_group = 교통량_group.sort_values('전체 추정교통량',ascending=False)
교통량_group.head()

,link_id,읍면동명,도로등급,링크길이,전체 추정교통량,승용차 추정교통량,버스 추정교통량,화물차 추정교통량,혼잡시간강도,혼잡빈도강도,혼잡시간_car,혼잡빈도_car
750,571250878,중앙동,101,1.235,106893.77,80488.27,5129.60,21275.90,65.29,54.94,69790.942433,58727.437238
751,571250880,중앙동,101,0.285,106888.21,80493.30,5127.94,21266.97,67.91,52.66,72587.783411,56287.331386
1017,571254992,대원동,101,1.238,104794.49,78556.90,5128.22,21109.36,59.66,49.10,62520.392734,51454.094590
1016,571254990,대원동,101,1.335,101620.94,76522.02,5156.78,19942.14,77.13,70.67,78380.231022,71815.518298
802,571251376,중앙동,101,0.265,100928.09,76283.58,5080.15,19564.36,73.85,63.64,74535.394465,64230.636476


In [ ]:
#### 상하행 한번 더 확인

In [44]:
a = 상세도로망[['link_id','up_lanes','dw_lanes']]

In [45]:
b = 교통량_merge[['link_id','상하행']]

In [47]:
ab = a.merge(b,on='link_id')
ab.head(100)
ab

,link_id,up_lanes,dw_lanes,상하행
0,478344684,1,0,상행
1,478344686,2,0,상행
2,478344690,1,0,상행
3,478344696,1,0,상행
4,478344700,1,0,상행
...,...,...,...,...
3274,571376981,2,0,상행
3275,571376982,2,0,상행
3276,571376983,2,0,상행
3277,900000166,1,0,상행


In [48]:
ab[ab.up_lanes=='0'] # 상행이 0인 곳은 없음!!! 즉 상행과 상/하행 동시에 있는양방향으로 나뉜다

,link_id,up_lanes,dw_lanes,상하행


In [38]:
# Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환
def line_string_to_coordinates(line_string):
    if isinstance(line_string, shapely.geometry.linestring.LineString):
        lon, lat = line_string.xy
        return [[x, y] for x, y in zip(lon, lat)]
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString):
        ret = []
        for i in range(len(line_string)):
            lon, lat = line_string[i].xy
            for x, y in zip(lon, lat):
                ret.append([x, y])
        return ret

In [39]:
상세도로망_c = 상세도로망.copy()

In [40]:
# 도로폭 정규화
상세도로망_c['norm_width'] = 상세도로망_c['width'].apply(int)/상세도로망_c['width'].apply(int).max()
# geometry 변환
상세도로망_c['geometry'] = 상세도로망_c['geometry'].apply(line_string_to_coordinates)

/opt/app-root/lib/python3.6/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [41]:
상세도로망_c = pd.DataFrame(상세도로망_c) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔야 함.
상세도로망_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 18 columns):
link_id       1875 non-null object
max_speed     1875 non-null object
road_name     1581 non-null object
road_no       1875 non-null object
road_rank     1875 non-null object
link_type     1875 non-null object
road_type     1875 non-null object
facil_name    33 non-null object
up_lanes      1875 non-null object
dw_lanes      1875 non-null object
oneway        1875 non-null object
length        1875 non-null object
width         1875 non-null object
car_lane      1875 non-null object
num_cross     1875 non-null object
barrier       1875 non-null object
geometry      1875 non-null object
norm_width    1875 non-null float64
dtypes: float64(1), object(17)
memory usage: 263.8+ KB


- 앞에서 상하행 분리했던 link_id로 상세도로망 데이터와 비로소 merge

In [42]:
교통량_c = 교통량_group.merge(상세도로망_c, on = 'link_id')

In [43]:
df = 교통량_c
df.columns

Index(['link_id', '읍면동명', '도로등급', '링크길이', '전체 추정교통량', '승용차 추정교통량', '버스 추정교통량',
       '화물차 추정교통량', '혼잡시간강도', '혼잡빈도강도', '혼잡시간_car', '혼잡빈도_car', 'max_speed',
       'road_name', 'road_no', 'road_rank', 'link_type', 'road_type',
       'facil_name', 'up_lanes', 'dw_lanes', 'oneway', 'length', 'width',
       'car_lane', 'num_cross', 'barrier', 'geometry', 'norm_width'],
      dtype='object')

In [44]:
str(df.columns.tolist)

"<bound method IndexOpsMixin.tolist of Index(['link_id', '읍면동명', '도로등급', '링크길이', '전체 추정교통량', '승용차 추정교통량', '버스 추정교통량',\n       '화물차 추정교통량', '혼잡시간강도', '혼잡빈도강도', '혼잡시간_car', '혼잡빈도_car', 'max_speed',\n       'road_name', 'road_no', 'road_rank', 'link_type', 'road_type',\n       'facil_name', 'up_lanes', 'dw_lanes', 'oneway', 'length', 'width',\n       'car_lane', 'num_cross', 'barrier', 'geometry', 'norm_width'],\n      dtype='object')>"

In [45]:
교통량_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1843 entries, 0 to 1842
Data columns (total 29 columns):
link_id       1843 non-null object
읍면동명          1843 non-null object
도로등급          1843 non-null int64
링크길이          1843 non-null float64
전체 추정교통량      1843 non-null float64
승용차 추정교통량     1843 non-null float64
버스 추정교통량      1843 non-null float64
화물차 추정교통량     1843 non-null float64
혼잡시간강도        1724 non-null float64
혼잡빈도강도        1724 non-null float64
혼잡시간_car      1724 non-null float64
혼잡빈도_car      1724 non-null float64
max_speed     1843 non-null object
road_name     1556 non-null object
road_no       1843 non-null object
road_rank     1843 non-null object
link_type     1843 non-null object
road_type     1843 non-null object
facil_name    33 non-null object
up_lanes      1843 non-null object
dw_lanes      1843 non-null object
oneway        1843 non-null object
length        1843 non-null object
width         1843 non-null object
car_lane      1843 non-null object
num_cross   

In [46]:
교통량_c.head()

,link_id,읍면동명,도로등급,링크길이,전체 추정교통량,승용차 추정교통량,버스 추정교통량,화물차 추정교통량,혼잡시간강도,혼잡빈도강도,...,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry,norm_width
0,571250878,중앙동,101,1.235,106893.77,80488.27,5129.60,21275.90,65.29,54.94,...,5,0,1,1.235,4,1,0,0,"[[127.08438444992294, 37.143317324961025], [12...",1.0
1,571250880,중앙동,101,0.285,106888.21,80493.30,5127.94,21266.97,67.91,52.66,...,5,0,1,0.285,4,1,0,0,"[[127.0859807099439, 37.15409227885504], [127....",1.0
2,571254992,대원동,101,1.238,104794.49,78556.90,5128.22,21109.36,59.66,49.10,...,5,0,1,1.238,4,1,0,0,"[[127.09157446032921, 37.13102674174111], [127...",1.0
3,571254990,대원동,101,1.335,101620.94,76522.02,5156.78,19942.14,77.13,70.67,...,5,0,1,1.335,4,1,0,0,"[[127.08480915187941, 37.141347445057605], [12...",1.0
4,571251376,중앙동,101,0.265,100928.09,76283.58,5080.15,19564.36,73.85,63.64,...,5,0,1,0.265,4,1,0,0,"[[127.08719726088344, 37.15728734851213], [127...",1.0


In [53]:
# 필요한것만 subset
# cols = ['link_id', '전체 추정교통량', '혼잡시간_car', '혼잡빈도_car', 'label', 'link_type', 'length', 'width', 'geometry', 'norm_width']
# df_subset = df[cols]

### 추정교통량 + 혼잡시간빈도 + 혼잡시간강도+ 상세도로망 >>> 교통량 관련 data 하나의 datatable로 병합 완료